In [ ]:
%load_ext autoreload
%autoreload 2
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from EventDriven.eventScheduler import *
from EventDriven.backtest import OptionSignalBacktest

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import json
with open('../input/profitable_weights.json', 'r') as f:
    weights = json.load(f)
ttrades_ = pd.read_csv('../input/profitable_trades.csv').iloc[:, 1:]

w_map = {x: w  * 0.75 for x, w in weights.items()}
ttrades_ = ttrades_[ttrades_['EntryTime'] != ttrades_['ExitTime']]
ttrades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,34.0,504.0,524.0,279.795877,253.010232,-910.711955,-0.095733,2023-07-05,2023-08-02,28 days,TSLA
1,28.0,504.0,526.0,192.240502,185.493749,-188.909083,-0.035095,2023-07-05,2023-08-04,30 days,AAPL
2,17.0,504.0,529.0,336.262811,322.011093,-242.279208,-0.042383,2023-07-05,2023-08-09,35 days,MSFT
3,48.0,504.0,536.0,87.043588,82.000000,-242.092217,-0.057943,2023-07-05,2023-08-18,44 days,AVGO
4,11.0,504.0,583.0,130.695846,122.257034,-92.826927,-0.064568,2023-07-05,2023-10-25,112 days,AMZN
5,178.0,504.0,753.0,42.282471,123.470001,14451.380412,1.920123,2023-07-05,2024-07-01,362 days,NVDA
7,5.0,522.0,556.0,332.810769,318.700396,-70.551863,-0.042398,2023-07-31,2023-09-18,49 days,HD
8,5.0,530.0,535.0,239.284572,224.549052,-73.677602,-0.061582,2023-08-10,2023-08-17,7 days,BA
9,5.0,530.0,535.0,53.784257,52.084613,-8.498219,-0.031601,2023-08-10,2023-08-17,7 days,WMT
10,38.0,535.0,580.0,226.851208,217.009995,-373.966096,-0.043382,2023-08-17,2023-10-20,64 days,TSLA


In [62]:
trades_ = ttrades_.copy()[:5]
msft_trade = pd.DataFrame([{"Size": 10.0, "EntryBar": 504, "ExitBar": 529.0, "EntryTime": "2023-08-09", "ExitTime": "2023-10-25", "Ticker": "MSFT"}])
trades_ = pd.concat([trades_, msft_trade], ignore_index=True)
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,34.0,504.0,524.0,279.795877,253.010232,-910.711955,-0.095733,2023-07-05,2023-08-02,28 days,TSLA
1,28.0,504.0,526.0,192.240502,185.493749,-188.909083,-0.035095,2023-07-05,2023-08-04,30 days,AAPL
2,17.0,504.0,529.0,336.262811,322.011093,-242.279208,-0.042383,2023-07-05,2023-08-09,35 days,MSFT
3,48.0,504.0,536.0,87.043588,82.000000,-242.092217,-0.057943,2023-07-05,2023-08-18,44 days,AVGO
4,11.0,504.0,583.0,130.695846,122.257034,-92.826927,-0.064568,2023-07-05,2023-10-25,112 days,AMZN
5,10.0,504.0,529.0,NaN,NaN,NaN,NaN,2023-08-09,2023-10-25,NaN,MSFT


In [63]:
#Backtest class 
evb_backtest = OptionSignalBacktest(trades_[trades_['Ticker'] == 'TSLA'])

In [64]:
evb_backtest.events.events

,type,symbol,signal_type,signal_id,max_contract_price,order_settings
datetime,,,,,,
2023-07-05,MARKET,NaN,NaN,NaN,NaN,NaN
2023-07-05,SIGNAL,TSLA,LONG,TSLA20230705LONG,NaN,NaN
2023-07-06,MARKET,NaN,NaN,NaN,NaN,NaN
2023-07-07,MARKET,NaN,NaN,NaN,NaN,NaN
2023-07-10,MARKET,NaN,NaN,NaN,NaN,NaN
2023-07-11,MARKET,NaN,NaN,NaN,NaN,NaN
2023-07-12,MARKET,NaN,NaN,NaN,NaN,NaN
2023-07-13,MARKET,NaN,NaN,NaN,NaN,NaN
2023-07-14,MARKET,NaN,NaN,NaN,NaN,NaN


# RUN BACKTEST

In [52]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable() 

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

Processing event: MARKET 2023-07-05 00:00:00
Processing event: SIGNAL 2023-07-05 00:00:00


## ***Signal ID: TSLA20230705LONG***

Using V2
Looks like our young fellow is targetting: 2023-07-05
Generating Data for TSLA 2024-06-21 00:00:00
Data Is_complete bool: False
In missing opttick but not in opttuick: 
['TSLA20240621C330', 'TSLA20240621C353.33', 'TSLA20240621C320', 'TSLA20240621C333.33', 'TSLA20240621C323.33', 'TSLA20240621C346.67', 'TSLA20240621C360', 'TSLA20240621C343.33', 'TSLA20240621C366.67', 'TSLA20240621C326.67', 'TSLA20240621C356.67', 'TSLA20240621C336.67', 'TSLA20240621C350', 'TSLA20240621C340', 'TSLA20240621C363.33']
Time taken to update cache: 0.02247333526611328
I'm proud of you, we are finally done
Actually! We are not done yet. We need to get the spot prices for the requested date
Now, my dear friend, we are done
Order Produced: {'result': 'SUCCESSFUL', 'data': {'long': ['TSLA20240621C266.67'], 'short': ['TSLA20240621C300'], 'trade_id': '&L:TSLA20240621C266.67&S:TSLA20240621C300', 'close': 14.749999999999993}}
Position ID: &L:TSLA20240621C266.67&S:TSLA20240621C300
Calculating Position Greeks
202

In [53]:
evb_backtest.portfolio.events.events

,type,symbol,signal_type,signal_id,max_contract_price,order_settings,order_type,cash,quantity,direction,position,exchange,fill_cost,market_value,slippage,commission
datetime,,,,,,,,,,,,,,,,
2023-07-05,MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-05,FILL,TSLA,NaN,TSLA20230705LONG,NaN,NaN,NaN,NaN,67.0,BUY,"{'long': ['TSLA20240621C266.67'], 'short': ['T...",ARCA,989.327226,988.953366,0.703366,0.37386
2023-07-05,ORDER,TSLA,NaN,TSLA20230705LONG,NaN,NaN,MKT,1000.0,NaN,BUY,"{'long': ['TSLA20240621C266.67'], 'short': ['T...",NaN,NaN,NaN,NaN,NaN
2023-07-05,SIGNAL,TSLA,LONG,TSLA20230705LONG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-06,MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-07,MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-10,MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-11,MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-12,MARKET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
(evb_backtest.portfolio.options_data['TSLA20240621C266.67'] - evb_backtest.portfolio.options_data['TSLA20240621C300']).iloc[-5:]

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-07-27 16:00:00,12.90,12.90,13.00,12.60,-261,-17,12.10,-118,11.95,12.025,11.903842
2023-07-28 16:00:00,12.25,13.10,12.90,13.65,-94,-44,13.30,46,13.45,13.375,13.497033
2023-07-31 16:00:00,11.59,13.55,12.80,14.05,-361,35,13.40,36,13.75,13.575,13.581487
2023-08-01 16:00:00,13.30,13.05,12.78,12.41,-151,-37,12.80,68,13.00,12.900,13.008403
2023-08-02 16:00:00,13.70,12.50,12.20,12.59,682,-26,12.05,-92,11.95,12.000,11.948024


In [55]:
evb_backtest.portfolio.new_trades['&L:TSLA20240621C266.67&S:TSLA20240621C300'].entries()

,datetime,uid,price,quantity,symbol,commission,market_value,slippage,total_cost,aux_cost,direction,transaction_type
0,2023-07-05,&L:TSLA20240621C266.67&S:TSLA20240621C300_TSLA...,1476.607800,67,TSLA,37.386,98895.336633,70.336633,98932.722633,107.722633,BUY,BUY
1,2023-07-18,&L:TSLA20240621C266.67&S:TSLA20240621C300_TSLA...,1537.385503,33,TSLA,18.414,50752.135615,-67.864385,50733.721615,86.278385,SELL,SELL
2,2023-08-02,&L:TSLA20240621C266.67&S:TSLA20240621C300_TSLA...,1288.700922,34,TSLA,18.972,43834.803333,-25.196667,43815.831333,44.168667,SELL,SELL


In [56]:
evb_backtest.portfolio.new_trades['&L:TSLA20240621C266.67&S:TSLA20240621C300'].stats

,trade_id,symbol,entry_date,exit_date,avg_entry_price,total_entry_commission,total_entry_slippage,total_entry_quantity,total_aux_entry_cost,avg_entry_cost,avg_exit_price,total_exit_commission,total_exit_slippage,total_exit_quantity,total_aux_exit_cost,avg_exit_cost,closed_quantity,closed_pnl,total_commission,total_slippage,total_aux_cost,open_quantity,unrealized_pnl,total_pnl,return_pct,duration_days
0,&L:TSLA20240621C266.67&S:TSLA20240621C300,TSLA,2023-07-05,2023-08-02,1476.6078,37.386,70.336633,67,107.722633,98932.722633,1411.187357,37.386,-93.061053,67,130.447053,47223.150427,67,-4383.169686,74.772,-22.72442,52.04758,0,0,-4383.169686,-0.044305,28


In [57]:
evb_backtest.portfolio.new_trades['&L:TSLA20240621C266.67&S:TSLA20240621C300'].is_closed()

True

In [59]:
evb_backtest.portfolio.trades

We dey here


,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions,SignalID
0,TSLA,-12589.760889,-0.127256,1476.6078,37.386,70.336633,98895.336633,989.327226,107.722633,1288.700922,18.972,-25.196667,43834.803333,438.158313,44.168667,67,2023-07-05,2023-08-02,28,&L:TSLA20240621C266.67&S:TSLA20240621C300,TSLA20230705LONG


In [38]:
evb_backtest.portfolio.aggregate()

We dey here


Start                                                    2023-07-05 00:00:00
End                                                      2023-08-02 00:00:00
Duration                                                    28 days 00:00:00
Exposure Time [%]                                                      100.0
Equity Final [$]                                                    87234.16
Equity Peak [$]                                                104691.181111
Return [%]                                                        -12.620508
Buy & Hold Return [%]                                              -8.198812
CAGR [%]                                                          -82.772146
Volatility Ann. [%]                                                97.881314
Sharpe Ratio                                                       -1.309656
Sortino Ratio                                                      -1.485101
Skew                                                               -0.568658

In [195]:
pd.DataFrame(evb_backtest.portfolio.unprocessed_signals)

""


In [107]:
evb_backtest.portfolio.trades[evb_backtest.portfolio.trades['Positions'] == "&L:TSLA20240621C283.33&S:TSLA20240621C400"]

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions,SignalID


In [108]:
evb_backtest.portfolio.get_all_positions()

long               short  \
datetime   symbol                                              
2023-07-05 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
           TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-06 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-07 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-10 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-11 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-12 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-13 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-14 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-17 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-18 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-19 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-20 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-21 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-24 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-25 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-26 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-27 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-28 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   
2023-07-31 TSLA    [TSLA20240621C266.67]  [TSLA20240621C300]   

                                                    trade_id   close  \
datetime   symbol                                                      
2023-07-05 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  14.750   
           TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  14.750   
2023-07-06 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  14.750   
2023-07-07 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  14.025   
2023-07-10 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  13.450   
2023-07-11 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  13.450   
2023-07-12 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  13.425   
2023-07-13 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  13.775   
2023-07-14 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  14.325   
2023-07-17 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  15.400   
2023-07-18 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  15.400   
2023-07-19 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  15.475   
2023-07-20 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  15.400   
2023-07-21 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  12.675   
2023-07-24 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  13.450   
2023-07-25 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  13.450   
2023-07-26 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  13.375   
2023-07-27 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  12.725   
2023-07-28 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  12.025   
2023-07-31 TSLA    &L:TSLA20240621C266.67&S:TSLA20240621C300  13.575   

                   quantity  market_value  TSLA20230705LONG  
datetime   symbol                                            
2023-07-05 TSLA          67       98825.0  TSLA20230705LONG  
           TSLA          67       98825.0  TSLA20230705LONG  
2023-07-06 TSLA          67       98825.0  TSLA20230705LONG  
2023-07-07 TSLA          67       93967.5  TSLA20230705LONG  
2023-07-10 TSLA          67       90115.0  TSLA20230705LONG  
2023-07-11 TSLA          67       90115.0  TSLA20230705LONG  
2023-07-12 TSLA          67       89947.5  TSLA20230705LONG  
2023-07-13 TSLA          67       92292.5  TSLA20230705LONG  
2023-07-14 TSLA          67       95977.5  TSLA20230705LONG  
2023-07-17 TSLA          67      103180.0  TSLA20230705LONG  
2023-07-18 TSLA          67      103180.0  TSLA20230705LONG  
2023-07-19 TSLA          67      103682.5  TSLA20230705LONG  
2023-07-20 TSLA          67      103180.0  TSLA20230705LONG  
2023-07-21 TSLA          67       84922.5  TSLA20230705LONG  
2023-07-24 TSLA          67       90115.0  TSLA202

In [109]:
pd.DataFrame([evb_backtest.portfolio.new_trades["&L:TSLA20240621C283.33&S:TSLA20240621C400"].aggregate()])

KeyError: '&L:TSLA20240621C283.33&S:TSLA20240621C400'

In [ ]:
evb_backtest.portfolio.new_trades["&L:TSLA20240621C283.33&S:TSLA20240621C400"].entries()

,datetime,uid,price,quantity,symbol,commission,market_value,slippage,total_cost,aux_cost,direction,transaction_type
0,2023-07-05,&L:TSLA20240621C283.33&S:TSLA20240621C400_TSLA...,3425.952655,2,TSLA,1.116,6850.789310,5.789310,6851.905310,6.905310,BUY,BUY
1,2023-08-02,&L:TSLA20240621C283.33&S:TSLA20240621C400_TSLA...,2546.980849,2,TSLA,1.116,5095.077698,-4.922302,5093.961698,6.038302,SELL,SELL


In [ ]:
trades =evb_backtest.portfolio.trades.drop_duplicates(subset=['SignalID'], keep='last')
trades

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions,SignalID
0,TSLA,-1758.187424,-0.256609,3425.803559,1.116,5.491118,6850.491118,68.516071,6.607118,2546.709847,1.116,-5.464305,5094.535695,50.934197,6.580305,2,2023-07-05,2023-08-02,28,&L:TSLA20240621C283.33&S:TSLA20240621C400,TSLA20230705LONG
1,AAPL,-987.036652,-0.264068,1868.904114,1.116,3.692228,3736.692228,37.378082,4.808228,1375.385788,1.116,-4.112423,2751.887577,27.507716,5.228423,2,2023-07-05,2023-08-04,30,&L:AAPL20240621C195&S:AAPL20240621C270,AAPL20230705LONG
2,MSFT,-834.910357,-0.266301,3135.218016,0.558,2.160016,3134.660016,31.352180,2.718016,2300.307659,0.558,-4.134341,2300.865659,23.003077,4.692341,1,2023-07-05,2023-08-09,35,&L:MSFT20240621C345&S:MSFT20240621C450,MSFT20230705LONG
3,AMZN,-483.823998,-0.316582,1528.274524,0.558,2.716524,1527.716524,15.282745,3.274524,1044.450526,0.558,-1.491474,1045.008526,10.444505,2.049474,1,2023-07-05,2023-10-25,112,&L:AMZN20240621C132.5&S:AMZN20240621C185,AMZN20230705LONG
5,AVGO,-1585.489899,-0.241469,3283.009707,1.116,4.903414,6564.903414,65.660194,6.019414,2490.264758,1.116,-8.354485,4981.645515,49.805295,9.470485,2,2023-07-05,2023-08-18,44,&L:AVGO20240119C900&S:AVGO20240119C1000,AVGO20230705LONG
6,BA,-175.407686,-0.245680,713.969026,0.558,0.911026,713.411026,7.139690,1.469026,538.561340,0.558,-0.880660,539.119340,5.385613,1.438660,1,2023-08-10,2023-08-17,7,&L:BA20240621C260&S:BA20240621C280,BA20230810LONG
7,WMT,-281.398480,-0.339162,414.843638,1.116,0.571275,828.571275,8.296873,1.687275,274.144398,1.116,-0.595205,549.404795,5.482888,1.711205,2,2023-08-16,2023-08-17,1,&L:WMT20231215C165&S:WMT20231215C180,WMT20230810LONG
8,TSLA,-474.304591,-0.084021,2822.527182,1.116,3.938364,5643.938364,56.450544,5.054364,2585.374887,1.116,-8.134227,5171.865773,51.707498,9.250227,2,2023-08-17,2023-10-20,64,&L:TSLA20240920C220&S:TSLA20240920C310,TSLA20230817LONG
9,AAPL,-510.410136,-0.250059,2041.162305,0.558,2.104305,2040.604305,20.411623,2.662305,1530.752169,0.558,-1.689831,1531.310169,15.307522,2.247831,1,2023-08-31,2023-09-07,7,&L:AAPL20240920C190&S:AAPL20240920C260,AAPL20230831LONG
10,WMT,-214.059682,-0.464728,460.613205,0.558,0.555205,460.055205,4.606132,1.113205,246.553523,0.558,-0.388477,247.111523,2.465535,0.946477,1,2023-09-14,2023-09-29,15,&L:WMT20240119C170&S:WMT20240119C200,WMT20230913LONG


In [ ]:
events_df = evb_backtest.events.events
events_df = events_df[events_df['symbol'] == 'JNJ']
events_df

,type,symbol,signal_type,signal_id,max_contract_price,order_settings,reason,order_type,quantity,direction,position,exchange,fill_cost,market_value,slippage,commission
datetime,,,,,,,,,,,,,,,,
2023-07-31,SIGNAL,JNJ,CLOSE,JNJ20230731LONG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-31,SIGNAL,JNJ,LONG,JNJ20230731LONG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-31,FILL,JNJ,NaN,JNJ20230731LONG,NaN,NaN,NaN,NaN,7.0,BUY,"{'long': ['JNJ20240621C160'], 'short': ['JNJ20...",ARCA,88.974588,88.935528,0.175528,0.03906
2023-07-31,ORDER,JNJ,NaN,JNJ20230731LONG,NaN,NaN,NaN,MKT,7.0,BUY,"{'long': ['JNJ20240621C160'], 'short': ['JNJ20...",NaN,NaN,NaN,NaN,NaN
2024-05-22,ROLL,JNJ,LONG,JNJ20230731LONG,NaN,NaN,NaN,NaN,NaN,NaN,"{'position': {'long': ['JNJ20240621C160'], 'sh...",NaN,NaN,NaN,NaN,NaN


In [ ]:
events_df = evb_backtest.events.events
events_df = events_df[events_df['symbol'] == 'NVDA']
events_df

,type,symbol,signal_type,signal_id,max_contract_price,order_settings,reason,order_type,quantity,direction,position,exchange,fill_cost,market_value,slippage,commission
datetime,,,,,,,,,,,,,,,,
2023-07-05,SIGNAL,NVDA,LONG,NVDA20230705LONG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-05,ORDER,NVDA,NaN,NVDA20230705LONG,NaN,NaN,NaN,MKT,1.0,BUY,"{'long': ['NVDA20240621C410'], 'short': ['NVDA...",NaN,NaN,NaN,NaN,NaN
2023-07-05,FILL,NVDA,NaN,NVDA20230705LONG,NaN,NaN,NaN,NaN,1.0,BUY,"{'long': ['NVDA20240621C410'], 'short': ['NVDA...",ARCA,48.668809,48.663229,0.063229,0.00558
2024-05-22,ORDER,NVDA,NaN,NVDA20230705LONG,NaN,NaN,NaN,MKT,3.0,BUY,"{'long': ['NVDA20250620C1050'], 'short': ['NVD...",NaN,NaN,NaN,NaN,NaN
2024-05-22,ROLL,NVDA,LONG,NVDA20230705LONG,NaN,NaN,NaN,NaN,NaN,NaN,"{'position': {'long': ['NVDA20240621C410'], 's...",NaN,NaN,NaN,NaN,NaN
2024-05-22,SIGNAL,NVDA,CLOSE,NVDA20230705LONG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-22,ORDER,NVDA,NaN,NVDA20230705LONG,NaN,NaN,NaN,MKT,1.0,SELL,"{'long': ['NVDA20240621C410'], 'short': ['NVDA...",NaN,NaN,NaN,NaN,NaN
2024-05-22,FILL,NVDA,NaN,NVDA20230705LONG,NaN,NaN,NaN,NaN,1.0,SELL,"{'long': ['NVDA20240621C410'], 'short': ['NVDA...",ARCA,139.143749,139.149329,-0.100671,0.00558
2024-05-22,SIGNAL,NVDA,LONG,NVDA20230705LONG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
evb_backtest.clean_run(trades=new_trades)

Processing event: MARKET
Processing event: SIGNAL
Buy Details
Position: {'long': ['TSLA20240621C260'], 'short': ['TSLA20240621C370'], 'trade_id': '&L:TSLA20240621C260&S:TSLA20240621C370', 'close': 39.22500000000001}, Date: 2023-07-05, Signal: SignalEvent type:LONG, symbol=TSLA, date:2023-07-05 00:00:00, Order Settings=None,Max Contract Price:None , signal_id:TSLA20230705LONG
Max Contract Price: 50.0, Cash at Hand: 90.0
Order Quantity 2 Cash at Hand 90.0 Close 39.22500000000001
Processing event: SIGNAL
Buy Details
Position: {'long': ['AAPL20240621C175'], 'short': ['AAPL20240621C255'], 'trade_id': '&L:AAPL20240621C175&S:AAPL20240621C255', 'close': 30.55}, Date: 2023-07-05, Signal: SignalEvent type:LONG, symbol=AAPL, date:2023-07-05 00:00:00, Order Settings=None,Max Contract Price:None , signal_id:AAPL20230705LONG
Max Contract Price: 50.0, Cash at Hand: 90.0
Order Quantity 2 Cash at Hand 90.0 Close 30.55
Processing event: SIGNAL
Buy Details
Position: {'long': ['MSFT20240621C315'], 'short'

KeyboardInterrupt: 

In [ ]:
evb_backtest.portfolio.trades_dict

{'&L:TSLA20240621C260&S:TSLA20240621C370': {'entry_price': 3926.3755205046727,
  'entry_date': Timestamp('2023-07-05 00:00:00'),
  'quantity': 2,
  'symbol': 'TSLA',
  'entry_commission': 1.1159999999999999,
  'entry_market_value': 7851.6350410093455,
  'entry_slippage': 6.6350410093434675,
  'total_entry_cost': 78.52751041009346,
  'auxilary_entry_cost': 7.751041009343467,
  'signal_id': 'TSLA20230705LONG',
  'trade_id': '&L:TSLA20240621C260&S:TSLA20240621C370',
  'exit_price': 3914.302534382505,
  'exit_date': Timestamp('2023-08-02 00:00:00'),
  'exit_commission': 1.1159999999999999,
  'exit_slippage': -15.278931234990978,
  'exit_market_value': 7829.72106876501,
  'total_exit_cost': 78.2860506876501,
  'auxilary_exit_cost': 16.394931234990977,
  'pnl': -24.1459722443351,
  'return_pct': -0.0061496848985120815,
  'duration_days': 28,
  'exit_method': 'sell'},
 '&L:AAPL20240621C175&S:AAPL20240621C255': {'entry_price': 3058.041873283369,
  'entry_date': Timestamp('2023-07-05 00:00:00')

In [ ]:
tsla = evb_backtest.portfolio.trades.loc['&L:TSLA20240621C260&S:TSLA20240621C370']
tsla['entry_date']

Timestamp('2023-07-05 00:00:00')

In [ ]:
(pd.to_datetime('2024-06-21') - pd.to_datetime('2024-06-21')).days

0

In [ ]:
tsla_option = evb_backtest.portfolio.get_option_data('TSLA20240621C260')
tsla_option = tsla_option[~tsla_option.index.duplicated(keep='last')]
tsla_option = tsla_option.loc[tsla['entry_date']]
tsla_option

Open                  73.530000
High                  74.000000
Low                   71.200000
Close                 74.000000
Volume                78.000000
Bid_size             183.000000
CloseBid              72.650000
Ask_size             396.000000
CloseAsk              73.950000
Midpoint              73.300000
Weighted_midpoint     73.539119
Name: 2023-07-05 00:00:00, dtype: float64

In [ ]:
evb_backtest.portfolio.get_all_positions()

long               short  \
datetime   symbol                                           
2023-07-05 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-06 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-07 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-10 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-11 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-12 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-13 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-14 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-17 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-18 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-19 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-20 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
           NVDA    [NVDA20240621C410]  [NVDA20240621C550]   
2023-07-21 TSLA    [TSLA20240621C260]  [TSLA20240621C370]   
           AAPL    [AAPL20240621C175]  [AAPL20240621C255]   
           MSFT    [MSFT20240621C315]  [MSFT20240621C450]   
           AMZN    [AMZN20240621C120]  [AMZN20240621C170]   
       

In [ ]:
import re
input_string = "&L:AMZN20240621C120&S:AMZN20240621C170"

# Regular expression to match the desired patterns
pattern = r'&[LS]:(\w+)'

# Find all matches
matches = re.findall(pattern, input_string)

# Output the results
for match in matches:
    print(match)

AMZN20240621C120
AMZN20240621C170
BA20240621C220
BA20240621C310


In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
HOLIDAY_SET = set(USFederalHolidayCalendar().holidays(start='2022-08-16', end='2022-08-16').strftime('%Y-%m-%d'))
HOLIDAY_SET

set()

In [ ]:
order_result = evb_backtest.risk_manager.OrderPicker.get_order('MSFT', '2023-09-05', 'C', 5.0,  {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .900,
   'dte': 365,
   'moneyness_width': 0.02},
   {'direction': 'short',
   'rel_strike': .80,
   'dte': 365,
   'moneyness_width': 0.02}],

 'name': 'vertical_spread'})

print(order_result)

{'result': 'TOO_ILLIQUID', 'data': None}


In [ ]:
stats.print_stats(30)
print(stream.getvalue())
with open('bactest_data.txt', 'w') as f:
  stream.seek(0)
  f.write(stream.read())
  f.flush()

         179946728 function calls (179230715 primitive calls) in 771.171 seconds

   Ordered by: cumulative time
   List reduced from 2996 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  771.173  385.587 c:\Users\Zino\python-playground\QuantTools\.venv\lib\site-packages\IPython\core\interactiveshell.py:3517(run_code)
        2    0.000    0.000  771.173  385.587 {built-in method builtins.exec}
        1    0.000    0.000  771.173  771.173 C:\Users\Zino\AppData\Local\Temp\ipykernel_28000\3226836942.py:1(<module>)
        1    0.028    0.028  771.173  771.173 C:\Users\Zino\python-playground\QuantTools\EventDriven\backtest.py:37(run)
       79    0.000    0.000  629.085    7.963 C:\Users\Zino\python-playground\QuantTools\EventDriven\portfolio.py:378(update_signal)
       79    0.003    0.000  629.083    7.963 C:\Users\Zino\python-playground\QuantTools\EventDriven\portfolio.py:267(generate_order)
       45 

In [ ]:
evb_backtest.portfolio.plot_portfolio()

AttributeError: 'OptionSignalPortfolio' object has no attribute '_trades'

In [ ]:
trades_df = evb_backtest.portfolio.get_trades().head(5)
trades_df

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions
0,TSLA,-10806.520059,-21.677784,498.506687,6.417,16.736810,11459.236810,11488.807620,-29.570810,390.441487,6.417,16.571196,8986.571196,9003.142393,-16.571196,23,2023-07-05,2023-08-02,28,&L:TSLA20240621C330&S:TSLA20240621C346.67
1,AAPL,3656.985131,7.479087,488.961456,3.627,-10.628066,6352.871934,6349.497867,3.374066,525.531308,3.627,-2.465999,6835.534001,6833.068001,2.465999,13,2023-07-05,2023-08-04,30,&L:AAPL20240621C230&S:AAPL20240621C280
2,MSFT,-10309.332567,-21.247498,485.202187,3.906,-1.075379,6788.924621,6795.661241,-6.736621,382.108862,3.906,-1.569939,5353.430061,5351.860122,1.569939,14,2023-07-05,2023-08-09,35,&L:MSFT20240621C355&S:MSFT20240621C365
3,AMZN,-5391.261254,-10.873383,495.821865,0.837,1.628594,1486.628594,1489.931189,-3.302594,441.909252,0.837,-0.935243,1326.564757,1325.629513,0.935243,3,2023-07-05,2023-10-25,112,&L:AMZN20240621C145&S:AMZN20240621C160


In [ ]:
evb_backtest.portfolio.get_weighted_holdings()s

In [ ]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

long                  short  \
datetime   symbol                                              
2023-07-05 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-06 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-07 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-10 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-11 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-12 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-13 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-14 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-17 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-18 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-19 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-20 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
           AAPL    [AAPL20240621C230]     [AAPL20240621C280]   
           MSFT    [MSFT20240621C355]     [MSFT20240621C365]   
           AMZN    [AMZN20240621C145]     [AMZN20240621C160]   
           NVDA    [NVDA20240621C440]     [NVDA20240621C450]   
2023-07-21 TSLA    [TSLA20240621C330]  [TSLA20240621C346.67]   
 

: 